```
From: https://github.com/ksatola
Version: 0.0.1

TODOs
1. 

```

# TensorFlow Extended Interactive Pipeline
Releases: https://github.com/tensorflow/tfx/blob/master/RELEASE.md

## Setup Jupyter Lab Python/Python environment
In the terminal/shell
```
#pip install virtualenv
#pip install virtualenvwrapper
#brew install pyenv-virtualenv

# https://realpython.com/intro-to-pyenv/#installing-pyenv
#Create a new virtualenv named tfx with Python 3.7.11
pyenv virtualenv 3.7.11 tfx3.7.11

pyenv versions

pyenv shell tfx3.7.11

pyenv which python
python -V

pip install tfx==0.22.0
#pip install apache-beam[interactive]
#sudo apt-get install libbz2-dev

#Making Jupyter Lab kernel linking to the proper python environment
#https://stackoverflow.com/questions/28831854/how-do-i-add-python3-kernel-to-jupyter-ipython/44072803#44072803
jupyter kernelspec list
jupyter --paths
/home/ksatola/.pyenv/versions/tfx3.7.11/bin/python -m ipykernel install --prefix=/home/ksatola/.local --name 'tfx3.7.11'
#jupyter kernelspec uninstall python3

pyenv shell tfx3.7.11 ; jupyter lab --no-browser
```

refs: 
- https://stackoverflow.com/questions/28831854/how-do-i-add-python3-kernel-to-jupyter-ipython/44072803#44072803
- https://ipython.readthedocs.io/en/6.5.0/install/kernel_install.html#kernels-for-different-environments
- https://www.alfredo.motta.name/create-isolated-jupyter-ipython-kernels-with-pyenv-and-virtualenv/
- https://innodesign.io/pythons-virtual-environments/
- https://janakiev.com/blog/jupyter-virtual-envs/

## TFX Pipeline

In [1]:
!python -V

pyenv: version `tfx' is not installed (set by /home/ksatola/git/MLOps/notebooks/.python-version)


In [2]:
!which python

/home/ksatola/.pyenv/shims/python


In [3]:
!tfx

pyenv: version `tfx' is not installed (set by /home/ksatola/git/MLOps/notebooks/.python-version)


In [4]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))

from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.5.1


ModuleNotFoundError: No module named '_bz2'

In [ ]:
base_dir = os.getcwd()
print(base_dir)

In [ ]:
os.chdir("/home/ksatola/git/MLOps")
!python utils/download_dataset.py

In [ ]:
os.getcwd()

In [ ]:
os.chdir("/home/ksatola/git/MLOps/notebooks")
os.getcwd()

In [ ]:
import os
import sys

#import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

#from tfx.utils.dsl_utils import external_input # deprecated
from tfx.components import CsvExampleGen

In [ ]:
context = InteractiveContext(pipeline_root='../tfx')

## ExampleGen

In [ ]:
# Remove any Jupyter .checkpoint files from the folder (there should be only one CSV file in the data folder)

base_dir = os.getcwd()
data_dir = "../data"
#examples = external_input(os.path.join(base_dir, data_dir))
data_file_name = 'consumer_complaints_with_narrative.csv'
example_gen = CsvExampleGen(input_base=data_dir)

context.run(example_gen)

In [ ]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

In [ ]:
context.show(statistics_gen.outputs['statistics'])

In [ ]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

In [ ]:
context.show(schema_gen.outputs['schema'])

In [ ]:
from tfx.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

In [ ]:
transform_file = os.path.join(base_dir, '../components/module.py')

In [ ]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=transform_file)
context.run(transform)

In [ ]:
trainer_file = os.path.join(base_dir, '../components/module.py')

In [ ]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor

TRAINING_STEPS = 1000
EVALUATION_STEPS = 100

trainer = Trainer(
    module_file=trainer_file,
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

### Load TensorBoard

In [ ]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri

In [ ]:
log_dir = os.path.join(model_artifact_dir, 'logs/')

%load_ext tensorboard
%tensorboard --logdir {log_dir} 

### Evaluate the model

In [ ]:
from tfx.components import ResolverNode
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing


model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

In [ ]:
# nb it always blesses on first run even if below threshold
import tensorflow_model_analysis as tfma

eval_config=tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
    slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
    metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(class_name='BinaryAccuracy'),
              tfma.MetricConfig(class_name='ExampleCount'),
              tfma.MetricConfig(class_name='AUC')
              ],
              thresholds={
                  'AUC':
                      tfma.config.MetricThreshold(
                          value_threshold=tfma.GenericValueThreshold(
                              lower_bound={'value': 0.65}),
                          change_threshold=tfma.GenericChangeThreshold(
                              direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                              absolute={'value': 0.01}))}
                          )])

In [ ]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)

In [ ]:
#NB TFMA visualizations will not run in Jupyter Lab
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

In [ ]:
tfma.view.render_slicing_metrics(tfma_result)

In [ ]:
from tfx.components.pusher.component import Pusher
from tfx.proto import pusher_pb2

_serving_model_dir = "../interactive-pipeline/serving_model_dir"

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

### Extra stuff

In [ ]:
!mkdir -p ../tfx-9Apr/serving_model_dir

In [ ]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
  
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
  
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix

# THEN REFRESH BROWSER PAGE!

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

In [ ]:
# Show data sliced by product
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='product')

In [ ]:
# fairness indicators direct from pipeline
# https://colab.research.google.com/github/tensorflow/fairness-indicators/blob/master/fairness_indicators/examples/Fairness_Indicators_Lineage_Case_Study.ipynb